
<img src="https://deep-hybrid-datacloud.eu/wp-content/uploads/sites/2/2018/01/logo.png" width="100">

<h1><center>DEEP-HybridDataCloud tutorial: Integrating a simple Keras application</center></h1>

This tutorial will guide you through the integration of a simple Deep Learning model (CIFAR10 image classification using Keras) with the API architecture. For more details you can check the [DEEP documentation repository](http://docs.deep-hybrid-datacloud.eu "DEEP docs")




If you want to have the general view of the model you can take a look at this [Model Notebook](https://github.com/laramaktub/IBERGRID-tutorial/blob/master/CIFAR10.ipynb "Notebook"). We will go in more detail through the tutorial.

For running this tutorial we are going to be using a [Docker container](https://www.docker.com). A Docker container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings. This means that the user doesn't have to worry about all these details and can focus only on the use/development of the application.

Take into account that you should have a git account for doing this tutorial. Make sure you have properly defined your **git identification variables:**

```console

$ git config --global user.email "you@example.com"
$ git config --global user.name "Your_Name"
```

First of all, we need to **install cookiecutter** which creates projects from project templates:
    

```console
$ pip install cookiecutter
```

Now that we have cookiecutter installed, we can generate the template for our project:

```console
$ cookiecutter https://github.com/indigo-dc/cookiecutter-data-science --checkout tutorial
```

The template for the project will be downloaded and you will be asked several questions. 

```
git_base_url [https://github.com/deephdc]: https://github.com/your_github_user
project_name [project_name]: cifar10
repo_name [cifar10]: cifar10
author_name [Your name (or your organization/company/team)]: your_name
author_email [Your email]: your_mail
description [A short description of the project.]: CIFAR10 classification app
app_version [Application version (expects X.Y.Z (Major.Minor.Patch))]: 1.0.0
Select open_source_license:
1 - MIT
2 - BSD-3-Clause
3 - No license file
Choose from 1, 2, 3 (1, 2, 3) [1]: 1
Select python_interpreter:
1 - python3
2 - python
Choose from 1, 2 (1, 2) [1]: 1
dockerhub_user [User account at hub.docker.com, e.g. 'deephdc' in https://hub.docker.com/u/deephdc]: your_docker_user
docker_baseimage [Base Docker image for Dockerfile, e.g. tensorflow/tensorflow]: tensorflow/tensorflow
baseimage_cpu_tag [CPU tag for the Base Docker image, e.g. 1.12.0-py3. Has to match python version!]: 1.10.0-py3
baseimage_gpu_tag [GPU tag for the Base Docker image, e.g. 1.12.0-gpu-py3. Has to match python version!]: 1.10.0-gpu-py3
```

This will create two project directories:

```  
~/cifar10
~/DEEP-OC-cifar10
```

Go to ``github.com/your_account`` and create the corresponding repositories: ``DEEP-OC-cifar10`` and ``cifar10``. 


Now enter in the directories you just created and do ``git push origin master`` in both created directories. This puts your initial code to github

You are now ready to run the Docker container. For the moment it will just contain an empty model.

Go to the ``~/DEEP-OC-cifar10`` directory and build the docker image:

```
sudo docker build -t cifar10docker .
      ```

This will generate a docker image that you can check by doing:

```console
$ sudo docker images
```

Now let's run the docker container by typing:

```console
$ sudo docker run -ti -p 5000:5000 -p 8888:8888 -p 6006:6006 cifar10docker /bin/bash
```

This will deploy the container. We can now type on the command line the following command to run the jupyter lab:

    
   

```console
$ /srv/.jupyter/run_jupyter.sh --allow-root
```

## SECOND PART OF TUTORIAL ##

This will generate an URL with a token. We can enter it in the web browser and we will now be running jupyter lab!

Now, in jupyter Lab, you can go to Terminal. You will be seeing the directory structure in your docker. You can launch the DEEPaaS API by typing:

```console
$ deepaas-run --listen-ip 0.0.0.0
```

The DEEPaaS API will be now running and accesible at ``your_VM_IP:5000``

You can easily check that right now there is no model loaded yet. For this, check the current metadata of the model and check the training part. You can see that there are no input arguments defined. 

Let's now include our **cifar10** model! 

```
requirements.txt
data/  
models/  
cifar10/dataset/make_dataset.py
cifar10/features/build_features.py  
cifar10/models/deepaas_api.py  
cifar10/config.py  
```

The file ``cifar10/config.py`` contains the input parameters that we want to give as input to our application via the web API. Open it and check that right now there is no parameter defined.

For this tutorial we want to have as parameters:
    * Learning rate
    * Number of epochs
    * Output directory
    
You can add this parameters by defining your train_args and your pred_args in the config.py file as:
    

In [1]:
train_args = { 'epochs': {'default': 1,
                        'help': 'Number of epochs',
                        'required': True,
                        'type': int
                        },

               'lrate': {'default':0.001,
                         'help': 'Initial learning rate value',
                         'required': True,
                         'type': float
                        },

               'outputpath': {'default': "/tmp",
                         'help': 'Path for saving the model',
                         'required': True
                        },
}

predict_args = { 'outputpath': {'default': "/tmp",
                          'help': 'Path for loading the model',
                          'required': True
                         },
}



Launch again the DEEPaaS API. You will now see these input parameters there!


In the ``requirements.txt`` file, one can include any additional package that may be needed to run the model. In our case, we only need to include one more line to this file for installing keras version 2.0.2. 

```
keras==2.0.2

```

Now let's download our model. For doing so, you  can go to the ``cifar10/models/`` folder and type:

```console
$ wget https://raw.githubusercontent.com/deephdc/DEEP-tutorial-cifar10/master/cifar10/models/cifar10_model.py
```

Open the ``cifar10_model.py`` file and take a look. You will see two functions called ``train_nn`` and ``predict_nn``. They contain our model both for training and for prediction respectively.

The ``deepaas_api.py`` is the script containing the functions that are going to be called from the DEEPaaS API. We will be editing only the functions:
    
    * get_metadata(): retrieves the model metadata
    * train(): launch the model training
    * predict_data(): predict on some image loaded by the user
        


Now you can download a filled version of the ``deepaas_api()`` file, go to the models folder and type:

```console
$ wget -O deepaas_api.py https://raw.githubusercontent.com/deephdc/DEEP-tutorial-cifar10/master/cifar10/models/deepaas_api.py
```

Open the file and check how the functions from our model are being imported:




    

```console
$ from cifar10.models.cifar10_model  import train_nn, predict_nn
```

Start by filling the metadata in the get_metadata() function. An example:

    meta = {
        'Name': "cifar10",
        'Version': "1.0.0",
        'Summary': "This is a simple implementation of cifar10 in keras",
        'Home-page': None,
        'Author': "Lara Lloret Iglesias",
        'Author-email': "lloret@ifca.unican.es",
        'License': "MIT",
    }

Now take a look at the ``predict_data()`` and ``train()`` functions.

The ``predict_data()`` has been modified in order to retrieve the loaded images and pass it to the

In [2]:
def predict_data(*args):
    """
    Function to make prediction on an uploaded file
    """
    outputpath=args[0]["outputpath"]
    thefile= args[0]['files'][0]
    thename= thefile.filename
    thepath= outputpath + "/" +thename
    
    thefile.save(thepath)
        
    img = image.load_img(thepath, target_size=(32,32))
    x= image.img_to_array(img)
   
    message=predict_nn(x,outputpath)
   
    return message


The ``train()`` function looks like this:

In [3]:
def train(train_args):
    
   # Train network
   # train_args : dict
   #     Json dict with the user's configuration parameters.
   #     Can be loaded with json.loads() or with yaml.safe_load()    

    train_nn(train_args['epochs'], train_args['lrate'],train_args['outputpath'])
   
    run_results = { "status": "SUCCESS",
                    "train_args": [],
                    "training": [],
                  }

    run_results["train_args"].append(train_args)

   
    return run_results

This retrieves the input arguments and passing them to the train_nn function from our model.

We are now ready to go!

Run the following command from the same directory where your requirements.txt is to install the cifar10 package with the new dependencies:

```console
$ python -m pip install -e .
```

Go to the terminal and launch the application again!

```console
$ deepaas-run --listen-ip 0.0.0.0
```

You can now play with the model directly from the DEEPaaS API.

    * Go to your browser and type your_VM_IP:5000
    * Run the get_metadata()
    * Save some truck picture from google and try to do some prediction (this should not work yet) 
    * Launch the training: try with 1 or 2 epochs, learning rate of 0.001 and the default output path ("/tmp")
    * Try prediction again with the truck picture
    * Feel free to train again with more samples, during more epochs, etc...

That was it! You have just integrated your first model in the DEEPaaS API!